In [116]:
import os
import json
import datetime as dt
import pandas as pd
import numpy as np
import sys
import requests
import datetime as dt

In [117]:
##get trailhead hotspot list

trailheadHotspots = pd.DataFrame()
trailheadHotspots = pd.read_csv('https://raw.githubusercontent.com/Tanag3r/trailheadDirect_birds/main/trailheadHotspots_Y2022_M1_D6.csv')

In [118]:
ebird_token = 'j6c7l80ga2ib'

##be nice to eBird, get a filtered list from all trailhead hotspots for testing
##the code below is for testing, interchange stopNames as needed
THfilters = ['HighSchool','EastSunsetWay']
THfiltersBool = trailheadHotspots.StopName.isin(THfilters)
selectHotspots = pd.DataFrame(trailheadHotspots[THfiltersBool])

In [135]:

##get recent notable observations

notableObs_current = []

##Really, really need to switch out this loop for a function
for locId in trailheadHotspots.itertuples():
##for locId in selectHotspots.itertuples():
    ebird_url = 'https://api.ebird.org/v2/data/obs/geo/recent/notable?'
    ebird_authHeader = {'X-eBirdApiToken': ebird_token}
    ebird_params = {
        'lat': str(locId.lat),
        'lng': str(locId.lng),
        'dist': str(2),
        'back': str(30),
        'detail': 'simple',
        'fmt': 'json'
    }
    ebird_request = requests.get(ebird_url,params=ebird_params,headers=ebird_authHeader)
    if ebird_request.status_code == requests.codes.ok:
        ebird_response = pd.DataFrame(ebird_request.json())
        ebird_response['StopName'] = locId.StopName
        notableObs_current.append(ebird_response)
    ebird_request.raise_for_status()

In [136]:
all_notableObs_current = pd.concat(notableObs_current)
##sanity check
all_notableObs_current.head(25)

,StopName,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,EastSunsetWay,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395
0,HighSchool,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2022-01-01 07:37,1.0,47.559756,-122.054982,True,True,False,S99862395


In [177]:
##get historical data on all notable observations in a +/- 30 day window

date_range = []

today = dt.date.today()
todayBack = today - dt.timedelta(30)
todayOut = today + dt.timedelta(30)
yearStart = todayBack.year - 4
yearStop = todayOut.year - 1
monthBack = todayBack.month
monthOut = todayOut.month

for i in range(yearStart,yearStop):
    days = pd.date_range(
        start=dt.date(i,monthBack,todayBack.day),
        end=dt.date(i+1,monthOut,todayOut.day)
    )
    date_range.append(days)


In [178]:
date_list = []
for sublist in date_range:
    for item in sublist:
        date_list.append(item)

In [179]:
##get historic notable observations at the given hotspots

histNotableObs_atLoc = []
notable_locList = list(all_notableObs_current['locId'])

for locId in notable_locList:
    for date in date_list:
        ##time.sleep(0.5)
        ymd = '{}/{}/{}'.format(date.year,date.month,date.day)
        ebirdHist_url = 'https://api.ebird.org/v2/data/obs/' + locId + '/historic/' + ymd
        ebirdHist_auth = {'X-eBirdApiToken': ebird_token}
        ebirdHist_params = {
            'fmt':'json',
            'detail':'simple'
        }
        ebirdHist_req = requests.get(ebirdHist_url,params=ebirdHist_params,headers=ebirdHist_auth)
        if ebirdHist_req.status_code == requests.codes.ok:
            ebirdHist_resp = pd.DataFrame(ebirdHist_req.json())
            ##ebirdHist_resp['StopName'] = pd.merge(ebirdHist_resp,all_notableObs_current[['StopName']],left_on='locId',right_on='locId',how='left')
            histNotableObs_atLoc.append(ebirdHist_resp)
        ebirdHist_req.raise_for_status()

In [180]:
all_histNotableObs = pd.concat(histNotableObs_atLoc)
##another sanity check
all_histNotableObs.head(15)

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
0,cangoo,Canada Goose,Branta canadensis,L321969,Lake Sammamish SP,2017-12-08 12:56,4.0,47.559756,-122.054982,True,False,False,S40996229
1,gadwal,Gadwall,Mareca strepera,L321969,Lake Sammamish SP,2017-12-08 12:56,8.0,47.559756,-122.054982,True,False,False,S40996229
2,mallar3,Mallard,Anas platyrhynchos,L321969,Lake Sammamish SP,2017-12-08 12:56,8.0,47.559756,-122.054982,True,False,False,S40996229
3,virrai,Virginia Rail,Rallus limicola,L321969,Lake Sammamish SP,2017-12-08 12:56,1.0,47.559756,-122.054982,True,False,False,S40996229
4,rethaw,Red-tailed Hawk,Buteo jamaicensis,L321969,Lake Sammamish SP,2017-12-08 12:56,1.0,47.559756,-122.054982,True,False,False,S40996229
5,norfli,Northern Flicker,Colaptes auratus,L321969,Lake Sammamish SP,2017-12-08 12:56,1.0,47.559756,-122.054982,True,False,False,S40996229
6,amecro,American Crow,Corvus brachyrhynchos,L321969,Lake Sammamish SP,2017-12-08 12:56,4.0,47.559756,-122.054982,True,False,False,S40996229
7,bkcchi,Black-capped Chickadee,Poecile atricapillus,L321969,Lake Sammamish SP,2017-12-08 12:56,3.0,47.559756,-122.054982,True,False,False,S40996229
8,ruckin,Ruby-crowned Kinglet,Corthylio calendula,L321969,Lake Sammamish SP,2017-12-08 12:56,1.0,47.559756,-122.054982,True,False,False,S40996229
9,foxspa,Fox Sparrow,Passerella iliaca,L321969,Lake Sammamish SP,2017-12-08 12:56,1.0,47.559756,-122.054982,True,False,False,S40996229


In [181]:
##filter for notables
SpeciesFilters = list(all_notableObs_current['speciesCode'])
SpeciesFilters_bool = all_histNotableObs.speciesCode.isin(SpeciesFilters)
select_obs = pd.DataFrame(all_histNotableObs[SpeciesFilters_bool])

select_obs.head()

,speciesCode,comName,sciName,locId,locName,obsDt,howMany,lat,lng,obsValid,obsReviewed,locationPrivate,subId
31,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2017-12-11 13:15,8.0,47.559756,-122.054982,True,True,False,S41037213
32,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-01-02 09:51,14.0,47.559756,-122.054982,True,True,False,S41557706
35,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-02-05 09:37,12.0,47.559756,-122.054982,True,True,False,S42519977
31,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2017-12-11 13:15,8.0,47.559756,-122.054982,True,True,False,S41037213
32,comred,Common Redpoll,Acanthis flammea,L321969,Lake Sammamish SP,2018-01-02 09:51,14.0,47.559756,-122.054982,True,True,False,S41557706


In [ ]:
##clean df
select_obs['obsDt'] = select_obs['obsDt'].astype('datetime64[ns]')

In [ ]:
##plot notables by locName